In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import time

In [2]:
# Le début des relevés est 2013. 

def get_all_data():
    base_url = "https://hubeau.eaufrance.fr/api/v1/vente_achat_phyto/achats/substances"
    params = {
        "type_territoire": "Département",
        "annee_min": 2012,
        "annee_max": 2013,
        "size": 1000,
        "page": 1
    }

    all_data = []

    print(" Accès à l'API en cours... Veuillez patienter.")
    
    start_time = time.time()
    
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data_page = response.json().get("data", [])
        
        if not data_page:
            break
        
        all_data.extend(data_page)
        params["page"] += 1

        
    end_time = time.time()
    duration = end_time - start_time

    print(f"Nombre de fichiers récupérés :{len(all_data)}")
    return pd.DataFrame(all_data)


In [3]:
df = get_all_data()

 Accès à l'API en cours... Veuillez patienter.
Nombre de fichiers récupérés :92880


In [4]:
df.head()

,amm,annee,achat_etranger,classification,classification_mention,code_cas,code_substance,code_territoire,fonction,libelle_substance,libelle_territoire,quantite,type_territoire,uri_substance,uri_territoire
0,8100462,2013,Non,Autre,-,8050-09-7,None,22,SA non phyto - Divers,resines (colophane),COTES-D'ARMOR,72.77725,Département,-,http://id.eaufrance.fr/dep/22
1,9700401,2013,Non,"T, T+, CMR",-,20427-59-2,5561,37,Fongicide,cuivre de l'hydroxyde de cuivre,INDRE-ET-LOIRE,52.50000,Département,http://id.eaufrance.fr/par/5561,http://id.eaufrance.fr/dep/37
2,8100444,2013,Non,Autre,-,24579-73-5,6398,62,Fongicide,propamocarbe,PAS-DE-CALAIS,3.61000,Département,http://id.eaufrance.fr/par/6398,http://id.eaufrance.fr/dep/62
3,2070133,2013,Non,N Organique,-,15299-99-7,1519,87,Herbicide,napropamide,HAUTE-VIENNE,4.50000,Département,http://id.eaufrance.fr/par/1519,http://id.eaufrance.fr/dep/87
4,2090018,2013,Non,N Organique,-,83164-33-4,1814,22,Herbicide,diflufenicanil,COTES-D'ARMOR,7.85240,Département,http://id.eaufrance.fr/par/1814,http://id.eaufrance.fr/dep/22


In [5]:
# Souhait de ne conserver que les substances étant classifiées CMR. Avant 2018, il était écrit T, T+, CMR
# et à partir de 2018, juste CMR. Je choisi donc de filtrer mon dataframe pour ne conserver que ces valeurs.
df_cmr = df[df["classification"].isin(["CMR", "T, T+, CMR"])]


In [6]:
df_cmr.head()

,amm,annee,achat_etranger,classification,classification_mention,code_cas,code_substance,code_territoire,fonction,libelle_substance,libelle_territoire,quantite,type_territoire,uri_substance,uri_territoire
1,9700401,2013,Non,"T, T+, CMR",-,20427-59-2,5561,37,Fongicide,cuivre de l'hydroxyde de cuivre,INDRE-ET-LOIRE,52.50,Département,http://id.eaufrance.fr/par/5561,http://id.eaufrance.fr/dep/37
22,2010426,2013,Non,"T, T+, CMR",-,98886-44-3,2744,29,Nématicide,fosthiazate,FINISTERE,2.00,Département,http://id.eaufrance.fr/par/2744,http://id.eaufrance.fr/dep/29
24,8800939,2013,Non,"T, T+, CMR",-,34123-59-6,1208,38,Herbicide,isoproturon,ISERE,125.00,Département,http://id.eaufrance.fr/par/1208,http://id.eaufrance.fr/dep/38
25,9300557,2013,Non,"T, T+, CMR",-,1897-45-6,1473,11,Fongicide,chlorothalonil,AUDE,3.75,Département,http://id.eaufrance.fr/par/1473,http://id.eaufrance.fr/dep/11
39,2110141,2013,Non,"T, T+, CMR",-,175013-18-0,2576,87,Fongicide,pyraclostrobine,HAUTE-VIENNE,0.20,Département,http://id.eaufrance.fr/par/2576,http://id.eaufrance.fr/dep/87


In [7]:
# Conservation des colonnes utiles pour la base de données. 
df_cmr = df_cmr[["amm", "annee", "classification_mention", "code_cas", "code_substance", "code_territoire", "fonction", "libelle_substance", "libelle_territoire", "quantite"]]

In [ ]:
df_cmr = df_cmr.reset_index(drop=True)

In [9]:
# Je vais maintenant supprimer les départements ayant le code territoire 0, 
# représentant une erreur dans la saisie. 
len(df_cmr)

10069

In [10]:
(df_cmr["code_territoire"] == "0").sum()

np.int64(0)

In [11]:
df_cmr = df_cmr[df_cmr["code_territoire"] != "0"]

In [12]:
len(df_cmr)

10069

In [13]:
df_cmr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10069 entries, 1 to 92876
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   amm                     10069 non-null  object 
 1   annee                   10069 non-null  int64  
 2   classification_mention  10069 non-null  object 
 3   code_cas                10069 non-null  object 
 4   code_substance          9442 non-null   object 
 5   code_territoire         10069 non-null  object 
 6   fonction                10069 non-null  object 
 7   libelle_substance       10069 non-null  object 
 8   libelle_territoire      10069 non-null  object 
 9   quantite                10069 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 865.3+ KB


In [16]:
# Je vais maintenant convertir les codes en int et changer les noms pour
# les rendre plus compréhensibles

colonnes_entiers = ["amm", "code_substance", "code_territoire"] 
for col in colonnes_entiers:
    df_cmr[col] = pd.to_numeric(df_cmr[col], errors="coerce").astype("Int64")


In [17]:
df_cmr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10069 entries, 0 to 10068
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   amm                     10069 non-null  Int64  
 1   annee                   10069 non-null  int64  
 2   classification_mention  10069 non-null  object 
 3   code_cas                10069 non-null  object 
 4   code_substance          9442 non-null   Int64  
 5   code_territoire         10040 non-null  Int64  
 6   fonction                10069 non-null  object 
 7   libelle_substance       10069 non-null  object 
 8   libelle_territoire      10069 non-null  object 
 9   quantite                10069 non-null  float64
dtypes: Int64(3), float64(1), int64(1), object(5)
memory usage: 816.3+ KB


In [19]:
df_cmr = df_cmr.rename(columns={
    "code_territoire": "num_département",
    "libelle_territoire": "département",
    "libelle_substance" : "nom_substance",
    "quantite": "quantite_en_kg",
})

df_cmr.head()

,amm,annee,classification_mention,code_cas,code_substance,num_département,fonction,nom_substance,département,quantite_en_kg
0,9700401,2013,-,20427-59-2,5561,37,Fongicide,cuivre de l'hydroxyde de cuivre,INDRE-ET-LOIRE,52.50
1,2010426,2013,-,98886-44-3,2744,29,Nématicide,fosthiazate,FINISTERE,2.00
2,8800939,2013,-,34123-59-6,1208,38,Herbicide,isoproturon,ISERE,125.00
3,9300557,2013,-,1897-45-6,1473,11,Fongicide,chlorothalonil,AUDE,3.75
4,2110141,2013,-,175013-18-0,2576,87,Fongicide,pyraclostrobine,HAUTE-VIENNE,0.20


Une fois les réglages effectués, on passe au script python qui va enregistrer en base de données. 
Lancer le script Insertion_data.py